In [1]:
pip install pandas

In [2]:
pip install sqlalchemy pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Import necessary libraries
import requests
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import psycopg2
import pyodbc

# Extraction Layer

In [6]:
#Extract data from URL
response = requests.get('https://en.wikipedia.org/wiki/List_of_largest_universities')

In [7]:
soup = BeautifulSoup(response.content)

In [8]:
table = soup.find('table', attrs={'class': 'sortable'}) #will give the first element that matches the criteria that you specify

In [9]:
trs = table.findAll('tr')

In [10]:
#To print the text of html items
for item in trs[0].findAll('th'):
    print(item.text.strip()) #strip() to remove extra spaces

Country
University
Founded
Type
Enrollment
Ref


In [11]:
columns = []
for item in trs[0].findAll('th'):
    print(item.text.strip())
    columns.append(item.text.strip())

Country
University
Founded
Type
Enrollment
Ref


In [12]:
columns

['Country', 'University', 'Founded', 'Type', 'Enrollment', 'Ref']

In [13]:
columns = list(map(lambda x: x.text.strip(), trs[0].findAll('th')))
columns[-1] = "Link"

In [14]:
columns

['Country', 'University', 'Founded', 'Type', 'Enrollment', 'Link']

In [15]:
rows = trs[1:]

In [16]:
rows[0]

<tr>
<td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="500" data-file-width="700" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/36/Flag_of_Albania.svg/21px-Flag_of_Albania.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/36/Flag_of_Albania.svg/32px-Flag_of_Albania.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/3/36/Flag_of_Albania.svg/42px-Flag_of_Albania.svg.png 2x" width="21"/></span></span> </span><a href="/wiki/Albania" title="Albania">Albania</a>
</td>
<td><a href="/wiki/University_of_Tirana" title="University of Tirana">University of Tirana</a>
</td>
<td style="text-align: right;">1957
</td>
<td>Public
</td>
<td style="text-align: right;">35,000
</td>
<td>
</td></tr>

In [17]:
rows[0].findAll('td')

[<td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><span><img alt="" class="mw-file-element" data-file-height="500" data-file-width="700" decoding="async" height="15" src="//upload.wikimedia.org/wikipedia/commons/thumb/3/36/Flag_of_Albania.svg/21px-Flag_of_Albania.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/3/36/Flag_of_Albania.svg/32px-Flag_of_Albania.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/3/36/Flag_of_Albania.svg/42px-Flag_of_Albania.svg.png 2x" width="21"/></span></span> </span><a href="/wiki/Albania" title="Albania">Albania</a>
 </td>,
 <td><a href="/wiki/University_of_Tirana" title="University of Tirana">University of Tirana</a>
 </td>,
 <td style="text-align: right;">1957
 </td>,
 <td>Public
 </td>,
 <td style="text-align: right;">35,000
 </td>,
 <td>
 </td>]

In [18]:
list(map(lambda x: x.text.strip(), rows[0].findAll('td')))

['Albania', 'University of Tirana', '1957', 'Public', '35,000', '']

In [19]:
rows[0].findAll('td')[1].a.attrs['href'].lstrip('/')

'wiki/University_of_Tirana'

In [20]:
def extract_row(tr):
    row_soup_list = tr.findAll('td')
    row = list(map(lambda x: x.text.strip(),row_soup_list))
    if len(row)<6:
        row.append('')
    link = row_soup_list[1].a.attrs['href'].lstrip('/')
    row[-1] = f'https://en.wikipedia.org/{link}' if link else ''
    return row

In [21]:
extract_row(rows[35])

['Japan',
 'Nihon University',
 '1889',
 'Private',
 '70,667',
 'https://en.wikipedia.org/wiki/Nihon_University']

In [22]:
data = list(map(extract_row, rows))

In [23]:
with open ('universities.csv', 'w', newline='') as file:
    csvwriter = csv.writer(file)
    csvwriter.writerow(columns)
    csvwriter.writerows(data)

In [24]:
with open ('universities.json', 'w') as file:
    json.dump(data, file, indent=4)

# Transformation Layer

In [25]:
df = pd.DataFrame(data=data, columns=columns)

In [26]:
df

,Country,University,Founded,Type,Enrollment,Link
0,Albania,University of Tirana,1957,Public,"35,000",https://en.wikipedia.org/wiki/University_of_Ti...
1,Algeria,Constantine University,1978,Public,"85,000",https://en.wikipedia.org/wiki/List_of_universi...
2,Angola,Agostinho Neto University,1962,Public,"29,827",https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,Argentina,University of Buenos Aires,1821,Public,"311,175",https://en.wikipedia.org/wiki/University_of_Bu...
4,Australia,Monash University,1958,Public,"73,807",https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...
65,Turkey,Anadolu University,1958,Public,"1,969,733",https://en.wikipedia.org/wiki/Anadolu_University
66,United Kingdom,Open University,1969,Public,"253,075",https://en.wikipedia.org/wiki/Open_University
67,United States,Texas A&M University,1876,Public,"73,284",https://en.wikipedia.org/wiki/Texas_A%26M_Univ...
68,Uruguay,University of the Republic,1949,Public,"144,108",https://en.wikipedia.org/wiki/University_of_th...


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Country     70 non-null     object
 1   University  70 non-null     object
 2   Founded     70 non-null     object
 3   Type        70 non-null     object
 4   Enrollment  70 non-null     object
 5   Link        70 non-null     object
dtypes: object(6)
memory usage: 3.4+ KB


In [28]:
df['Founded'].values

array(['1957', '1978', '1962', '1821', '1958', '1365', '1992', '1921',
       '1425', '1949', '1982', '1970', '1888', '1827', '1960', '1867',
       '1940', '1669', '1928', '1538', '1950', '1632', '1640', '1939',
       '1998', '1837', '1676', '1937', '1911', '1985', '1984', '1982',
       '1834', '1956', '1303', '1889', '1970', '1999', '1579', '2000',
       '1956', '1551', '1959', '1632', '1883', '1943', '1996', '2002',
       '1551', '1904', '1816', '1288', '1991', '1992', '1976', '1905',
       '1919', '1873', '1972', '1972', '1666', '1833[34]', '1928', '1971',
       '1960', '1958', '1969', '1876', '1949', '1721'], dtype=object)

In [29]:
#To extract only the first four digits for the year (as you can see 1833 has extra characters)
df['Founded'] = df['Founded'].str.extract("(\d{4})")

In [30]:
df['Founded'].values

array(['1957', '1978', '1962', '1821', '1958', '1365', '1992', '1921',
       '1425', '1949', '1982', '1970', '1888', '1827', '1960', '1867',
       '1940', '1669', '1928', '1538', '1950', '1632', '1640', '1939',
       '1998', '1837', '1676', '1937', '1911', '1985', '1984', '1982',
       '1834', '1956', '1303', '1889', '1970', '1999', '1579', '2000',
       '1956', '1551', '1959', '1632', '1883', '1943', '1996', '2002',
       '1551', '1904', '1816', '1288', '1991', '1992', '1976', '1905',
       '1919', '1873', '1972', '1972', '1666', '1833', '1928', '1971',
       '1960', '1958', '1969', '1876', '1949', '1721'], dtype=object)

In [31]:
#To convert the Founded column datatype from string to integers (for years)
df['Founded'] = df['Founded'].astype(int)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Country     70 non-null     object
 1   University  70 non-null     object
 2   Founded     70 non-null     int64 
 3   Type        70 non-null     object
 4   Enrollment  70 non-null     object
 5   Link        70 non-null     object
dtypes: int64(1), object(5)
memory usage: 3.4+ KB


In [33]:
df[df['Link'].isnull()]

,Country,University,Founded,Type,Enrollment,Link


In [34]:
df['Enrollment'].values

array(['35,000', '85,000', '29,827', '311,175', '73,807', '91,000',
       '2,097,182', '25,500', '55,484', '30,866', '32,000', '215,000',
       '18 911', '84,000', '80,000', '53,581', '42,750', '72,480',
       '43,600', '170,530', '170,000', '14,000', '36,500', '350,000[8]',
       '130,000', '104,000', '124,000', '32,000', '13,782', '7,140,000',
       '1,045,665 (total) as per 2019/2020[14]\n311,028 (active) as per 4 November 2020 [15]',
       '1,000,000', '32,900', '26,023', '112,564', '70,667', '84,000',
       '41,833', '23,606', '10,373', '100,000', '349,515', '604,437',
       '31,186', '33,050', '50,000', '39,000', '165,000', '37,032',
       '68,249', '44,400', '48,100', '311,928', '200,000', '177,234',
       '38,300', '48,821', '328,179', '173,758', '260,079', '30,646',
       '26,356', '31,758', '525,000', '60,000+', '1,969,733', '253,075',
       '73,284', '144,108', '41,059'], dtype=object)

In [35]:
df['Enrollment'] = df['Enrollment'].str.replace(' ', ',')

In [36]:
df['Enrollment'] = df['Enrollment'].str.extract("(\d{1,3}(?:,\d{1,3})*)")

In [37]:
df['Enrollment'].values

array(['35,000', '85,000', '29,827', '311,175', '73,807', '91,000',
       '2,097,182', '25,500', '55,484', '30,866', '32,000', '215,000',
       '18,911', '84,000', '80,000', '53,581', '42,750', '72,480',
       '43,600', '170,530', '170,000', '14,000', '36,500', '350,000',
       '130,000', '104,000', '124,000', '32,000', '13,782', '7,140,000',
       '1,045,665', '1,000,000', '32,900', '26,023', '112,564', '70,667',
       '84,000', '41,833', '23,606', '10,373', '100,000', '349,515',
       '604,437', '31,186', '33,050', '50,000', '39,000', '165,000',
       '37,032', '68,249', '44,400', '48,100', '311,928', '200,000',
       '177,234', '38,300', '48,821', '328,179', '173,758', '260,079',
       '30,646', '26,356', '31,758', '525,000', '60,000', '1,969,733',
       '253,075', '73,284', '144,108', '41,059'], dtype=object)

In [38]:
df['Enrollment'] = df['Enrollment'].str.replace(',', '')

In [39]:
df.head()

,Country,University,Founded,Type,Enrollment,Link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University


In [40]:
#To convert the Enrolment column datatype from string to integer
df['Enrollment'] = df['Enrollment'].astype(int)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Country     70 non-null     object
 1   University  70 non-null     object
 2   Founded     70 non-null     int64 
 3   Type        70 non-null     object
 4   Enrollment  70 non-null     int64 
 5   Link        70 non-null     object
dtypes: int64(2), object(4)
memory usage: 3.4+ KB


In [42]:
df['Country'].values

array(['Albania', 'Algeria', 'Angola', 'Argentina', 'Australia',
       'Austria', 'Bangladesh', 'Belarus', 'Belgium',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria',
       'Canada', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Denmark',
       'Dominican Republic', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Guatemala', 'Hong Kong', 'Iceland', 'India',
       'Indonesia', 'Iran', 'Ireland', 'Israel', 'Italy', 'Japan',
       'Kenya', 'Kosovo', 'Lithuania', 'Macau', 'Malaysia', 'Mexico',
       'Nepal', 'Netherlands', 'New Zealand', 'North Macedonia', 'Norway',
       'Pakistan', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Romania',
       'Russia', 'Saudi Arabia', 'Singapore', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan',
       'Thailand', 'Trinidad and Tobago', 'Turkey', 'United Kingdom',
       'United States', 'Uruguay', 'Venezuela'], dtype=object)

In [43]:
df.to_csv('universities-clean.csv', index=False)

# Loading Layer

### Loading into MySQL using Docker

In [44]:
#Connect to the database
conn = pymysql.Connect(
    host= 'localhost',
    port= 3306,
    password= 'wikiuserpassword',
    user= 'wikiuser',
    database = 'wikidb',
    client_flag=pymysql.constants.CLIENT.MULTI_STATEMENTS #to read multiple statements in my sql file like DROP TABLE; CREATE TABLE;
)

RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods

In [45]:
#Create cursor to interact with the database
cursor = conn.cursor()

In [46]:
cursor.execute("SHOW TABLES;")

0

In [47]:
cursor.fetchall()

()

In [48]:
create_query = open('./sql/mysql_create_tables.sql')
create_query = create_query.read()
create_query

'DROP TABLE IF EXISTS universities;\n\nCREATE TABLE universities(\n        id BIGINT AUTO_INCREMENT PRIMARY KEY NOT NULL,\n        country VARCHAR (255) NOT NULL,\n        university_name VARCHAR (255) NOT NULL,\n        founded INT NOT NULL,\n        `type` VARCHAR (255) NOT NULL,\n        enrollment BIGINT NOT NULL,\n        link VARCHAR (255) NOT NULL,\n        CONSTRAINT unique_combination UNIQUE(country, university_name)\n);'

In [49]:
cursor.execute(create_query)

0

In [50]:
#Load data into tables already created
mysql_data = list(df.itertuples(index=None, name=None))

In [51]:
tuple(mysql_data)

(('Albania',
  'University of Tirana',
  1957,
  'Public',
  35000,
  'https://en.wikipedia.org/wiki/University_of_Tirana'),
 ('Algeria',
  'Constantine University',
  1978,
  'Public',
  85000,
  'https://en.wikipedia.org/wiki/List_of_universities_in_Algeria'),
 ('Angola',
  'Agostinho Neto University',
  1962,
  'Public',
  29827,
  'https://en.wikipedia.org/wiki/Agostinho_Neto_University'),
 ('Argentina',
  'University of Buenos Aires',
  1821,
  'Public',
  311175,
  'https://en.wikipedia.org/wiki/University_of_Buenos_Aires'),
 ('Australia',
  'Monash University',
  1958,
  'Public',
  73807,
  'https://en.wikipedia.org/wiki/Monash_University'),
 ('Austria',
  'University of Vienna',
  1365,
  'Public',
  91000,
  'https://en.wikipedia.org/wiki/University_of_Vienna'),
 ('Bangladesh',
  'National University, Bangladesh',
  1992,
  'Public',
  2097182,
  'https://en.wikipedia.org/wiki/National_University,_Bangladesh'),
 ('Belarus',
  'Belarusian State University',
  1921,
  'Public',

In [52]:
sql_engine = create_engine("mysql+pymysql://", creator=lambda: conn)

In [53]:
pd.read_sql("SELECT * FROM universities;", con=sql_engine)

,id,country,university_name,founded,type,enrollment,link


In [54]:
merge_query_file = open('./sql/mysql_upsert.sql')
merge_query = merge_query_file.read()
merge_query  

'INSERT INTO universities (country, university_name, founded, `type`, enrollment, link)\nVALUES (%s, %s, %s, %s, %s, %s) \nON DUPLICATE KEY UPDATE\n    founded=VALUES(founded),\n    `type`=VALUES(`type`),\n    enrollment=VALUES(enrollment),\n    link=VALUES(link);\n'

In [55]:
cursor.executemany(merge_query, mysql_data)

70

In [56]:
conn.commit()

In [57]:
pd.read_sql("SELECT * FROM universities;", con=sql_engine)

,id,country,university_name,founded,type,enrollment,link
0,1,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,2,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,3,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,4,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,5,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...,...
65,66,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
66,67,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
67,68,United States,Texas A&M University,1876,Public,73284,https://en.wikipedia.org/wiki/Texas_A%26M_Univ...
68,69,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...


### Loading Data into Postgres Database

In [58]:
df.head()

,Country,University,Founded,Type,Enrollment,Link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University


In [61]:
conn = psycopg2.connect(
    host = 'localhost',
    port = 5435,
    user = 'wikiuser',
    password = 'wikiuserpassword',
    database = 'wikidb'
)

In [62]:
cursor = conn.cursor()

In [68]:
create_query_file = open('./sql/postgres_create_tables.sql')
create_query = create_query_file.read()
create_query

'DROP TABLE IF EXISTS universities;\n\nCREATE TABLE universities(\n        id SERIAL PRIMARY KEY,\n        country VARCHAR (255) NOT NULL,\n        university_name VARCHAR (255) NOT NULL,\n        founded INT NOT NULL,\n        type VARCHAR (255) NOT NULL,\n        enrollment BIGINT NOT NULL,\n        link VARCHAR (255) NOT NULL,\n        CONSTRAINT unique_combination UNIQUE(country, university_name)\n);'

In [70]:
#conn.rollback()

In [71]:
cursor.execute(create_query)

In [72]:
conn.commit()

In [73]:
postgres_engine = create_engine("postgresql+psycopg2://", creator=lambda: conn)

In [74]:
pd.read_sql("SELECT * FROM universities;", con=postgres_engine)

,id,country,university_name,founded,type,enrollment,link


In [75]:
postgres_data = list(df.itertuples(name=None, index=None))

In [81]:
merge_query_file = open('./sql/postgres_upsert.sql')
merge_query = merge_query_file.read()
merge_query  

'INSERT INTO universities (country, university_name, founded, type, enrollment, link)\nVALUES (%s, %s, %s, %s, %s, %s) \nON CONFLICT(country, university_name) DO UPDATE\nSET\n    founded=EXCLUDED.founded,\n    type=EXCLUDED.type,\n    enrollment=EXCLUDED.enrollment,\n    link=EXCLUDED.link;'

In [84]:
cursor.executemany(merge_query, postgres_data)

In [83]:
#conn.rollback()

In [85]:
conn.commit()

In [86]:
pd.read_sql("SELECT * FROM universities;", con=postgres_engine)

,id,country,university_name,founded,type,enrollment,link
0,1,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,2,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,3,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,4,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,5,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...,...
65,66,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
66,67,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
67,68,United States,Texas A&M University,1876,Public,73284,https://en.wikipedia.org/wiki/Texas_A%26M_Univ...
68,69,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...


### Loading Data into SQL Server

In [45]:
SERVER = 'localhost,1433'
USERNAME = 'sa'
PASSWORD = 'Wikipassword_007'

In [46]:
connectionString = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};UID={USERNAME};PWD={PASSWORD}'

In [47]:
conn = pyodbc.connect(connectionString)

In [48]:
cursor = conn.cursor()

In [49]:
cursor.execute("SELECT name FROM sys.databases;")

In [50]:
cursor.fetchall()

[('master',), ('tempdb',), ('model',), ('msdb',)]

In [55]:
conn.autocommit = True 

In [56]:
cursor.execute("CREATE DATABASE wikidb;")

In [57]:
cursor.execute("USE wikidb")

In [58]:
create_query_file = open('./sql/sqlserver_create_tables.sql')
create_query = create_query_file.read()
create_query

'DROP TABLE IF EXISTS universities;\n\nCREATE TABLE universities(\n        id BIGINT IDENTITY(1,1) PRIMARY KEY,\n        country NVARCHAR (255) NOT NULL,\n        university_name NVARCHAR (255) NOT NULL,\n        founded INT NOT NULL,\n        type NVARCHAR (255) NOT NULL,\n        enrollment BIGINT NOT NULL,\n        link NVARCHAR (255) NOT NULL,\n        CONSTRAINT unique_combination UNIQUE(country, university_name)\n);'

In [59]:
cursor.execute(create_query)

In [60]:
sqlserver_engine = create_engine("mssql+pyodbc://", creator=lambda: conn)

In [63]:
pd.read_sql("SELECT * FROM universities;", con=sqlserver_engine)

,id,country,university_name,founded,type,enrollment,link


In [65]:
# Creating a staging (temporary) table in the database instance
df.to_sql("#staging", index=False, con=sqlserver_engine)

70

In [66]:
pd.read_sql("SELECT * FROM #staging;", con=sqlserver_engine)

,Country,University,Founded,Type,Enrollment,Link
0,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...
65,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
66,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
67,United States,Texas A&M University,1876,Public,73284,https://en.wikipedia.org/wiki/Texas_A%26M_Univ...
68,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...


In [69]:
pd.read_sql("SELECT * FROM #staging WHERE country = 'France';", con=sqlserver_engine)

,Country,University,Founded,Type,Enrollment,Link
0,France,Centre national d'enseignement à distance,1939,Public,350000,https://en.wikipedia.org/wiki/Centre_national_...


In [75]:
merge_query_file = open('./sql/sqlserver_upsert.sql')
merge_query = merge_query_file.read()
merge_query

'MERGE INTO universities AS target\nUSING #staging AS source\nON (source.University = target.university_name AND source.Country = target.country)\nWHEN MATCHED THEN \n    UPDATE SET\n        target.founded = source.Founded,\n        target.type = source.Type,\n        target.enrollment = source.Enrollment,\n        target.link = source.Link\nWHEN NOT MATCHED BY TARGET THEN\n    INSERT (country, university_name, founded, type, enrollment, link)\n    VALUES (source.Country, source.University, source.Founded, source.Type, source.Enrollment, source.Link);'

In [76]:
cursor.execute(merge_query)

In [77]:
pd.read_sql("SELECT * FROM universities;", con=sqlserver_engine)

,id,country,university_name,founded,type,enrollment,link
0,1,Albania,University of Tirana,1957,Public,35000,https://en.wikipedia.org/wiki/University_of_Ti...
1,2,Algeria,Constantine University,1978,Public,85000,https://en.wikipedia.org/wiki/List_of_universi...
2,3,Angola,Agostinho Neto University,1962,Public,29827,https://en.wikipedia.org/wiki/Agostinho_Neto_U...
3,4,Argentina,University of Buenos Aires,1821,Public,311175,https://en.wikipedia.org/wiki/University_of_Bu...
4,5,Australia,Monash University,1958,Public,73807,https://en.wikipedia.org/wiki/Monash_University
...,...,...,...,...,...,...,...
65,66,Turkey,Anadolu University,1958,Public,1969733,https://en.wikipedia.org/wiki/Anadolu_University
66,67,United Kingdom,Open University,1969,Public,253075,https://en.wikipedia.org/wiki/Open_University
67,68,United States,Texas A&M University,1876,Public,73284,https://en.wikipedia.org/wiki/Texas_A%26M_Univ...
68,69,Uruguay,University of the Republic,1949,Public,144108,https://en.wikipedia.org/wiki/University_of_th...
